In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *
from setting import *

In [ ]:
gps_map = ccal.load_gps_map(PATH_DICT["gps_map.pickle.gz"])

In [ ]:
data_dicts = make_data_dicts()

In [ ]:
with open(HIGHLIGHT_JSON_FILE_PATH) as json_file:

    highlight = json.load(json_file)

for element_type in ("feature", "sample"):

    highlight[element_type] = {
        data_name: data_dict
        for data_name, data_dict in highlight[element_type].items()
        if data_name in data_dicts[element_type] and data_dict
    }

highlight

In [ ]:
signal = pd.read_table(
    PATH_DICT["feature_x_sample.context_signal.tsv"], index_col=0
).fillna(0)

for element_type in ("feature", "sample"):

    if element_type == "feature":

        axis = 1

    elif element_type == "sample":

        axis = 0

    element_signal_mean = signal.mean(axis=axis)

    element_signal_mean.name = "Mean"

    element_signal_std = signal.std(axis=axis)

    element_signal_std.name = "Standard Deviation"

    element_n_0 = (signal == 0).sum(axis=axis)

    element_n_0.name = "Number of 0"

    data_dicts[element_type]["signal"] = {
        "df": pd.DataFrame([element_signal_mean, element_signal_std, element_n_0]),
        "data_type": "continuous",
    }

    highlight[element_type]["signal"] = "All"

In [ ]:
for element_type in ("feature", "sample"):

    if element_type == "feature":

        element_alias = FEATURE_ALIAS

        w_or_h = "w"

        elements = gps_map.w_elements

        element_marker_size = GPS_MAP_W_ELEMENT_MARKER_SIZE

    elif element_type == "sample":

        element_alias = SAMPLE_ALIAS

        w_or_h = "h"

        elements = gps_map.h_elements

        element_marker_size = GPS_MAP_H_ELEMENT_MARKER_SIZE

    for data_name, data_indices in highlight[element_type].items():

        df = data_dicts[element_type][data_name]["df"]

        annotation_type = data_dicts[element_type][data_name]["data_type"]

        if data_indices == "All":

            data_indices = df.index.tolist()

        for index in df.index & set(data_indices):

            element_values = df.loc[index, elements]

            title = "GPS Map with {}<br>{}: {}".format(element_alias, data_name, index)

            if annotation_type == "binary" and element_values.sum() < 3:

                print("Skipping {} ...".format(title))

                continue

            file_name = ccal.make_file_name_from_str(
                "{}_{}.html".format(data_name, index)
            )

            html_file_path = "{}/{}".format(
                PATH_DICT["{}|hcc|map/".format(w_or_h)], file_name
            )

            map_plotly_directory_path = PATH_DICT["plotly|{}_map/".format(w_or_h)]

            if map_plotly_directory_path is None:

                plotly_file_path = None

            else:

                plotly_file_path = "{}/{}".format(map_plotly_directory_path, file_name)

            if annotation_type == "categorical":

                categorical_colorscale = ccal.make_colorscale(
                    n_category=element_values.dropna().unique().size, plot=False
                )

            else:

                categorical_colorscale = None

            gps_map.plot_gps_map(
                w_or_h,
                annotation_x_element=element_values.to_frame().T,
                annotation_std_maxs=(PLOT_STD,),
                annotation_types=(annotation_type,),
                annotation_colorscale=categorical_colorscale,
                title=title,
                element_marker_size=element_marker_size,
                html_file_path=html_file_path,
                plotly_file_path=plotly_file_path,
            )

            if element_type == "feature":

                element_states = gps_map.w_element_states

            elif element_type == "sample":

                element_states = gps_map.h_element_states

            if element_states is not None:

                state_series = element_values.groupby(by=element_states)

                names = tuple(
                    "{} State {}".format(element_alias, state)
                    for state in state_series.groups.keys()
                )

                html_file_path = "{}/{}".format(
                    PATH_DICT["{}|hcc|comparison/".format(w_or_h)], file_name
                )

                if annotation_type == "continuous":

                    ccal.plot_violin_or_box(
                        (series for group, series in state_series),
                        names=names,
                        violin_or_box="box",
                        title=title,
                        html_file_path=html_file_path,
                    )

                elif annotation_type == "binary":

                    ccal.plot_bar(
                        ((name,) for name in names),
                        ((sum_,) for sum_ in state_series.sum()),
                        names=names,
                        orientation="v",
                        title=title,
                        xaxis_title="{} State".format(element_alias),
                        yaxis_title="N",
                        html_file_path=html_file_path,
                    )